<a href="https://colab.research.google.com/github/K-107/exercise/blob/main/DETR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os
import cv2
import torch
from torch import nn
from torchvision.models import resnet50
import tensorflow as tf

In [47]:
path = '/content/drive/MyDrive/음식인식/img_girl.jpg'
img = cv2.imread(path, cv2.IMREAD_ANYCOLOR)
np_img = np.array(img)

In [48]:
np_img.shape, img.shape

((800, 1200, 3), (800, 1200, 3))

In [49]:
np_img = np.expand_dims(img, axis=0)
np_img.shape

(1, 800, 1200, 3)

In [50]:
tf_img = tf.transpose(np_img, perm=[0, 3, 1, 2])
np_img = tf_img.numpy()
np_img.shape

(1, 3, 800, 1200)

In [58]:
torch_img = torch.tensor(np_img)
torch_img.shape

torch.Size([1, 3, 800, 1200])

In [59]:
torch_img

tensor([[[[ 44,  29,  22,  ..., 133, 138, 143],
          [ 44,  36,  30,  ..., 134, 134, 135],
          [ 41,  41,  34,  ..., 131, 129, 129],
          ...,
          [125, 119, 111,  ...,  94,  91,  93],
          [117,  99,  91,  ...,  98,  91,  89],
          [ 83,  76,  70,  ...,  99,  89,  82]],

         [[ 84,  69,  60,  ..., 155, 158, 162],
          [ 84,  76,  68,  ..., 156, 154, 154],
          [ 78,  78,  69,  ..., 153, 151, 151],
          ...,
          [147, 141, 131,  ..., 138, 137, 139],
          [139, 121, 111,  ..., 143, 137, 135],
          [105,  98,  90,  ..., 144, 135, 128]],

         [[ 49,  34,  24,  ..., 130, 135, 141],
          [ 49,  41,  32,  ..., 131, 131, 133],
          [ 44,  44,  35,  ..., 128, 127, 127],
          ...,
          [158, 152, 142,  ...,  91,  90,  92],
          [151, 132, 122,  ...,  94,  90,  88],
          [117, 110, 101,  ...,  95,  88,  81]]]], dtype=torch.uint8)

In [66]:
torch_img = torch_img.type(torch.LongTensor)
torch_img = torch_img/255
torch_img

tensor([[[[0.1725, 0.1137, 0.0863,  ..., 0.5216, 0.5412, 0.5608],
          [0.1725, 0.1412, 0.1176,  ..., 0.5255, 0.5255, 0.5294],
          [0.1608, 0.1608, 0.1333,  ..., 0.5137, 0.5059, 0.5059],
          ...,
          [0.4902, 0.4667, 0.4353,  ..., 0.3686, 0.3569, 0.3647],
          [0.4588, 0.3882, 0.3569,  ..., 0.3843, 0.3569, 0.3490],
          [0.3255, 0.2980, 0.2745,  ..., 0.3882, 0.3490, 0.3216]],

         [[0.3294, 0.2706, 0.2353,  ..., 0.6078, 0.6196, 0.6353],
          [0.3294, 0.2980, 0.2667,  ..., 0.6118, 0.6039, 0.6039],
          [0.3059, 0.3059, 0.2706,  ..., 0.6000, 0.5922, 0.5922],
          ...,
          [0.5765, 0.5529, 0.5137,  ..., 0.5412, 0.5373, 0.5451],
          [0.5451, 0.4745, 0.4353,  ..., 0.5608, 0.5373, 0.5294],
          [0.4118, 0.3843, 0.3529,  ..., 0.5647, 0.5294, 0.5020]],

         [[0.1922, 0.1333, 0.0941,  ..., 0.5098, 0.5294, 0.5529],
          [0.1922, 0.1608, 0.1255,  ..., 0.5137, 0.5137, 0.5216],
          [0.1725, 0.1725, 0.1373,  ..., 0

In [75]:
class DETR(nn.Module):
  def __init__(self, num_classes, hidden_dim, nheads, num_encoder_layers, num_decoder_layers):
    super().__init__()
    # We take only convolutional layers from ResNet-50 model
    self.backbone = nn.Sequential(*list(resnet50(pretrained=True).children())[:-2])
    self.conv = nn.Conv2d(2048, hidden_dim, 1)
    self.transformer = nn.Transformer(hidden_dim, nheads, num_encoder_layers, num_decoder_layers)
    self.linear_class = nn.Linear(hidden_dim, num_classes + 1)
    self.linear_bbox = nn.Linear(hidden_dim, 4)
    self.query_pos = nn.Parameter(torch.rand(100, hidden_dim))
    self.row_embed = nn.Parameter(torch.rand(50, hidden_dim // 2))
    self.col_embed = nn.Parameter(torch.rand(50, hidden_dim // 2))

  def forward(self, inputs):
    x = self.backbone(inputs)
    h = self.conv(x)
    H, W = h.shape[-2:]
    pos = torch.cat([
    self.col_embed[:W].unsqueeze(0).repeat(H, 1, 1),
    self.row_embed[:H].unsqueeze(1).repeat(1, W, 1),
    ], dim=-1).flatten(0, 1).unsqueeze(1)
    h = self.transformer(pos + h.flatten(2).permute(2, 0, 1),
    self.query_pos.unsqueeze(1))
    return self.linear_class(h), self.linear_bbox(h).sigmoid()


detr = DETR(num_classes=91, hidden_dim=256, nheads=8, num_encoder_layers=6, num_decoder_layers=6)
detr.eval()
inputs = torch.randn(1, 3, 800, 1200)
inputs1 = torch_img
logits, bboxes = detr(inputs)
logits1, bboxes1 = detr(inputs1)


In [76]:
logits.shape

torch.Size([100, 1, 92])

In [77]:
logits1.shape

torch.Size([100, 1, 92])

In [91]:
a = torch.ones(5)
b = a.numpy()
print(b)

[1. 1. 1. 1. 1.]


In [94]:
a

tensor([1., 1., 1., 1., 1.])

In [92]:
logits1

tensor([[[-1.7012e-01,  2.6034e-01, -5.1403e-02,  ...,  5.5652e-02,
           7.0462e-01, -3.1343e-01]],

        [[-4.0364e-03,  2.0445e-01, -1.0340e-01,  ...,  1.2523e-01,
           6.4167e-01, -3.9646e-01]],

        [[-2.1653e-02,  3.6273e-01, -1.1103e-02,  ...,  1.0918e-01,
           6.5414e-01, -4.2067e-01]],

        ...,

        [[-6.6616e-02,  1.4370e-01, -9.6459e-02,  ...,  9.6160e-02,
           6.3207e-01, -4.1516e-01]],

        [[-1.2988e-01,  1.5655e-01,  1.3377e-02,  ...,  6.2119e-02,
           6.8481e-01, -4.5978e-01]],

        [[-2.2720e-01,  2.4774e-01, -4.3460e-02,  ...,  5.3901e-04,
           6.4076e-01, -3.6974e-01]]], grad_fn=<AddBackward0>)

In [102]:
torch.argmax(logits1, dim=0)

tensor([[ 6,  2, 93, 43, 71, 22, 42, 33,  2, 69, 92, 45, 23, 91, 88, 92, 71, 92,
          8,  8, 50, 70, 61,  5, 84, 28, 46,  7, 17, 25, 42, 12, 56, 51, 45, 19,
         37, 47, 92, 27,  3, 65, 55, 58, 42, 52, 52, 85, 96,  8, 33, 23, 86, 36,
          6, 91, 38, 36, 66, 68, 37, 15, 95, 59,  6, 42, 94, 33, 27, 92, 92, 70,
         81,  1, 64, 29, 28, 46, 28, 86, 36, 66, 25, 21, 95, 65, 81, 85, 20, 19,
         77, 46]])

In [101]:
torch.argmax(logits1, dim=2)

tensor([[31],
        [31],
        [31],
        [31],
        [31],
        [31],
        [31],
        [31],
        [31],
        [31],
        [31],
        [31],
        [31],
        [31],
        [31],
        [31],
        [31],
        [31],
        [31],
        [73],
        [31],
        [31],
        [31],
        [31],
        [31],
        [31],
        [31],
        [31],
        [31],
        [31],
        [31],
        [31],
        [31],
        [73],
        [31],
        [31],
        [31],
        [31],
        [31],
        [31],
        [31],
        [31],
        [31],
        [31],
        [31],
        [31],
        [31],
        [31],
        [31],
        [31],
        [31],
        [31],
        [31],
        [31],
        [31],
        [31],
        [31],
        [31],
        [31],
        [31],
        [31],
        [31],
        [31],
        [31],
        [31],
        [31],
        [31],
        [31],
        [31],
        [31],
        [31],
      

In [78]:
bboxes.shape

torch.Size([100, 1, 4])

In [79]:
bboxes1.shape

torch.Size([100, 1, 4])

In [82]:
# COCO classes
CLASSES = [
    'N/A', 'person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus',
    'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'N/A',
    'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse',
    'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'N/A', 'backpack',
    'umbrella', 'N/A', 'N/A', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis',
    'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove',
    'skateboard', 'surfboard', 'tennis racket', 'bottle', 'N/A', 'wine glass',
    'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich',
    'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake',
    'chair', 'couch', 'potted plant', 'bed', 'N/A', 'dining table', 'N/A',
    'N/A', 'toilet', 'N/A', 'tv', 'laptop', 'mouse', 'remote', 'keyboard',
    'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'N/A',
    'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier',
    'toothbrush'
]

In [83]:
len(CLASSES)

91

In [ ]:
import matplotlib.pyplot as plt

def plot_results(pil_img, prob=None, boxes=None):
    plt.figure(figsize=(16,10))
    plt.imshow(pil_img)
    ax = plt.gca()
    colors = COLORS * 100
    if prob is not None and boxes is not None:
      for p, (xmin, ymin, xmax, ymax), c in zip(prob, boxes.tolist(), colors):
          ax.add_patch(plt.Rectangle((xmin, ymin), xmax - xmin, ymax - ymin,
                                    fill=False, color=c, linewidth=3))
          cl = p.argmax()
          text = f'{CLASSES[cl]}: {p[cl]:0.2f}'
          ax.text(xmin, ymin, text, fontsize=15,
                  bbox=dict(facecolor='yellow', alpha=0.5))
    plt.axis('off')
    plt.show()